In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import numpy as np
import scipy as sp
import model

In [8]:
"""
Just for quick debugging of diffusion map implementation
"""

def dMap_inefficient(data):
       # Form distance matrix
       d_mat = sp.spatial.distance_matrix(data, data)
       
       # Set diameter of dataset to 5%
       epsilon = 0.05 * np.max(d_mat)
       
       # Form un-normalized kernel matrix
       w_mat = np.exp(-np.square(d_mat) / epsilon)
       
       # Form normalized kernel matrix
       p_mat = np.diag(w_mat.sum(axis=1, dtype='float'))
       p_mat_inv = np.linalg.inv(p_mat)
       k_mat = np.matmul(p_mat_inv, np.matmul(w_mat, p_mat_inv))
       
       # Form normalized symmetric matrix
       q_mat = np.diag(k_mat.sum(axis=1, dtype='float'))
       q_mat_inv_sqr = np.sqrt(np.linalg.inv(q_mat))
       t_mat = np.matmul(q_mat_inv_sqr, np.matmul(k_mat, q_mat_inv_sqr))
       
def dMap_sparse(data, max_dist=1.0):
       # Form distance matrix
       kd_tree = sp.spatial.KDTree(data)
       d_mat = kd_tree.sparse_distance_matrix(kd_tree, max_distance=max_dist)
       d_mat = d_mat.toarray()
       
       # Set diameter of dataset to 5%
       epsilon = 0.05 * np.max(d_mat)
       
       # Form un-normalized kernel matrix
       w_mat = np.exp(-np.square(d_mat) / epsilon)
       
       # Form normalized kernel matrix
       p_mat = sp.sparse.diags(w_mat.sum(axis=1, dtype='float'), format="csc")
       p_mat_inv = sp.sparse.linalg.inv(p_mat)
       k_mat = p_mat_inv.dot(sp.sparse.csc_matrix.dot(w_mat, p_mat_inv))
       
       # Form normalized symmetric matrix
       q_mat = sp.sparse.diags(k_mat.sum(axis=1, dtype='float'), format="csc")
       q_mat_inv_sqr = np.sqrt(sp.sparse.linalg.inv(q_mat))
       t_mat = q_mat_inv_sqr.dot(sp.sparse.csc_matrix.dot(k_mat, q_mat_inv_sqr))
       

In [9]:
# Debug model
data = [[0.1, 0.1, 0.2],
       [0.3, 0.4, 0.5],
       [0.6, 0.7, 0.8]]

diffMap = model.DiffusionMap()
diffMap.fit(data)

print(diffMap.phis)
print(diffMap.lambdas)
print(diffMap.dMap)

[[-0.53518966  0.62313298]
 [-0.2749342  -0.77662707]
 [ 0.80318335  0.14933478]]
[0.94559063 0.77024381]
[[-0.50607033  0.47996432]
 [-0.2599752  -0.59819219]
 [ 0.75948265  0.11502419]]
